In [ ]:
# Generate a list of fully qualified variable names:
legs = ("FL", "FR", "HL", "HR")
ldofs = ("HX", "HY", "KN")
dof_vars = ("q", "q_d", "qd", "qd_d", "tau", "tau_d")
leg_vars_xyz = ("ffe.f", "p_ft_rt_b", "v_ft_rt_b")
leg_vars = ("foot_force_z", "contact_state.in_contact", "contact_state.td_detected")

varnames = []
base = "lemo"
for leg in legs:
    lvar = '.'.join([base, leg])
    for leg_var in leg_vars:
        varnames.append('.'.join([lvar, leg_var]))
    for lvc in leg_vars_xyz:
        for x in ["x", "y", "z"]:
            varnames.append('.'.join([lvar, lvc, x]))
    for ld in ldofs:
        dv = '.'.join([lvar, ld])
        
        for dvar in dof_vars:
            varnames.append('.'.join([dv, dvar]))

for var in sorted(varnames):
    print(var)
        

In [ ]:
import pprint

def dict_from_var_list(varlist):
    ''' This takes a list of variable names representing a flattened data structure:
          e.g. foo.bar.baz.item1
               foo.bar.baz.item2
               foo.bar.qux.item1
               etc...
        And parses them into a dictionary, where each key is represented by the
        substrings that result from splitting on the '.'
        The final value is the flattened variable name.
    '''
    # Create an empty dictionary to store our tree view of data
    vartree = dict()

    # Walk each variable in the list
    for var in sorted(varlist):
        # Split the variable on the '.' as this is the delimiter of the flattened structure
        parts = var.split('.')
        # Grab a reference the top level dictionary
        d = vartree
        # For each part of the variable, check to see if the current part is the last part
        # (it needs special handling)
        # Otherwise, check to see if the key already exists, if not, add the key and assign
        # it an empty dictionary. Then grab a reference to that new dictionary for the
        # next iteration in the loop.
        for p in parts:
            if p == parts[-1]:
                d[p] = var
                continue
            elif p not in d:
                d[p] = dict()
            d = d[p]
            
    return vartree
    
treeview = dict_from_var_list(varnames)
pprint.pprint(treeview)

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, \
                            QVBoxLayout, QTreeWidget, QTreeWidgetItem
from PyQt5.QtCore import Qt

def fill_item(w, d):
    #print("Walking dictionary")
    for k in d.keys():
        #print(f"For key={k}, type={type(d[k])}")
        if type(d[k]) is dict:
            ti = QTreeWidgetItem(w, [k])
            fill_item(ti, d[k])
        else:
            ti = QTreeWidgetItem(w, [k, str(d[k])])
        ti.setExpanded(True)

class Application(QMainWindow):
    def __init__(self):
        QMainWindow.__init__(self)
        
        main_widget = QWidget()
        self.setCentralWidget(main_widget)
        
        layout = QVBoxLayout(main_widget)
        
        tw = QTreeWidget()
        tw.setHeaderLabels(['Var', 'Value'])
        tw.setAlternatingRowColors(True)

        fill_item(tw.invisibleRootItem(), treeview)
 
        layout.addWidget(tw)
     

main_thread = QApplication([])
app = Application()
app.show()
main_thread.exec()